In [42]:
import numpy as np
import pandas as pd
import pickle
from pathlib import Path
from datasets import Dataset
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import matplotlib.pyplot as plt

# Path to your GEO TXT file
file_path = "unprocessed_datasets/GSE37642-GPL96_series_matrix.txt"

# Step 1: Read the file as text to see the header lines
with open(file_path, 'r') as f:
    lines = f.readlines()

# Header lines usually start with '!'
header_lines = [line.strip() for line in lines if line.startswith('!')]

# Example: print first few header lines
for line in header_lines[30:]:
    print(line)


!Series_contact_address	"Marchioninistr. 15"
!Series_contact_city	"Munich"
!Series_contact_zip/postal_code	"81377"
!Series_contact_country	"Germany"
!Series_supplementary_file	"ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE37nnn/GSE37642/suppl/GSE37642_RAW.tar"
!Series_supplementary_file	"ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE37nnn/GSE37642/suppl/GSE37642_Survival_data.txt.gz"
!Series_platform_id	"GPL96"
!Series_platform_id	"GPL97"
!Series_platform_id	"GPL570"
!Series_platform_taxid	"9606"
!Series_sample_taxid	"9606"
!Series_relation	"BioProject: https://www.ncbi.nlm.nih.gov/bioproject/PRJNA162567"
!Sample_title	"AML_patient_1"	"AML_patient_2"	"AML_patient_3"	"AML_patient_4"	"AML_patient_5"	"AML_patient_6"	"AML_patient_7"	"AML_patient_8"	"AML_patient_9"	"AML_patient_10"	"AML_patient_11"	"AML_patient_12"	"AML_patient_13"	"AML_patient_14"	"AML_patient_15"	"AML_patient_16"	"AML_patient_17"	"AML_patient_18"	"AML_patient_19"	"AML_patient_20"	"AML_patient_21"	"AML_patient_22"	"AML_patient_23"	

In [43]:
import pandas as pd
from io import StringIO
#GET EXPRESSION MATRIX

#file_path = "GSE13159_series_matrix.txt"

# Read all lines
with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()

# Find data section
data_start = None
data_end = None
for i, line in enumerate(lines):
    if '!platform_table_begin' in line:
        data_start = i + 1
    elif '!platform_table_end' in line:
        data_end = i
        break

# Slice table section
table_lines = lines[data_start:data_end]

# Detect header line ("ID_REF" usually marks it)
header_line_idx = None
for i, line in enumerate(table_lines):
    if line.startswith("ID_REF"):
        header_line_idx = i
        break

# Combine from header line onward
table_text = ''.join(table_lines[header_line_idx:])

# Parse with pandas (robust to quotes and irregular whitespace)
df = pd.read_csv(
    StringIO(table_text),
    sep="\t",
    quotechar='"',
    engine="python",
    comment="!",
)

print("Expression data loaded successfully!")
print("Shape:", df.shape)
print(df.head())
#ID_REF = gene/probe ID
#GSM... = sample ID


Expression data loaded successfully!
Shape: (22277, 423)
      ID_REF  GSM923596  GSM923597  GSM923598  GSM923599  GSM923600  \
0  1007_s_at       8.37       8.11       8.11       7.70       7.94   
1    1053_at       7.34       7.69       7.86       7.82       7.62   
2     117_at       8.25       7.10       6.93       6.60       7.11   
3     121_at       9.83       9.95       9.57       9.45       9.77   
4  1255_g_at       4.91       4.96       4.96       4.76       4.80   

   GSM923601  GSM923602  GSM923603  GSM923604  ...  GSM924008  GSM924009  \
0       8.51       8.88       8.04       9.03  ...       7.87       8.29   
1       7.90       7.18       8.00       7.21  ...       7.81       7.84   
2       7.08       6.81       7.20       6.86  ...       6.94       7.20   
3       9.55      10.26       9.67      10.19  ...       9.43       9.66   
4       4.75       5.01       4.88       5.29  ...       4.83       4.89   

   GSM924010  GSM924011  GSM924012  GSM924013  GSM924014  G

In [ ]:
#GET METADATA
# Step 1: Read all lines
with open(file_path, 'r') as f:
    lines = f.readlines()

# Step 2: Extract metadata lines only
metadata_lines = [l.strip() for l in lines if l.startswith("!Sample_")]

# Step 3: Turn them into a dictionary {key: [values]}
metadata_dict = {}
for line in metadata_lines:
    parts = line.split("\t")
    key = parts[0].replace("!Sample_", "")
    values = [v.strip('"') for v in parts[1:]]  # remove quotes
    metadata_dict[key] = values

# Step 4: Convert to DataFrame
metadata_df = pd.DataFrame(metadata_dict)

print("Metadata shape:", metadata_df.shape)
#print(metadata_df.head(5))
print(list(metadata_df.columns))
print(metadata_df["title"].tail(5)) #column with AML labels, source_name_ch1 for GSE51082 and GSE9476

Metadata shape: (422, 31)
['title', 'geo_accession', 'status', 'submission_date', 'last_update_date', 'type', 'channel_count', 'source_name_ch1', 'organism_ch1', 'characteristics_ch1', 'treatment_protocol_ch1', 'molecule_ch1', 'extract_protocol_ch1', 'label_ch1', 'label_protocol_ch1', 'taxid_ch1', 'hyb_protocol', 'scan_protocol', 'data_processing', 'platform_id', 'contact_name', 'contact_email', 'contact_department', 'contact_institute', 'contact_address', 'contact_city', 'contact_zip/postal_code', 'contact_country', 'supplementary_file', 'data_row_count', 'relation']
417    AML_patient_418
418    AML_patient_419
419    AML_patient_420
420    AML_patient_421
421    AML_patient_422
Name: title, dtype: object


In [46]:
#TYPES OF LEUKEMIA IN DATASET
# List unique entries in the characteristics_ch1 column
unique_chars = metadata_df["title"].unique() #column with AML labels, source_name_ch1 for GSE51082 and GSE9476

# Print them nicely
for val in unique_chars:
    print(val)

AML_patient_1
AML_patient_2
AML_patient_3
AML_patient_4
AML_patient_5
AML_patient_6
AML_patient_7
AML_patient_8
AML_patient_9
AML_patient_10
AML_patient_11
AML_patient_12
AML_patient_13
AML_patient_14
AML_patient_15
AML_patient_16
AML_patient_17
AML_patient_18
AML_patient_19
AML_patient_20
AML_patient_21
AML_patient_22
AML_patient_23
AML_patient_24
AML_patient_25
AML_patient_26
AML_patient_27
AML_patient_28
AML_patient_29
AML_patient_30
AML_patient_31
AML_patient_32
AML_patient_33
AML_patient_34
AML_patient_35
AML_patient_36
AML_patient_37
AML_patient_38
AML_patient_39
AML_patient_40
AML_patient_41
AML_patient_42
AML_patient_43
AML_patient_44
AML_patient_45
AML_patient_46
AML_patient_47
AML_patient_48
AML_patient_49
AML_patient_50
AML_patient_51
AML_patient_52
AML_patient_53
AML_patient_54
AML_patient_55
AML_patient_56
AML_patient_57
AML_patient_58
AML_patient_59
AML_patient_60
AML_patient_61
AML_patient_62
AML_patient_63
AML_patient_64
AML_patient_65
AML_patient_66
AML_patient_67
AML_

In [47]:
with open("metadata_types_GSE37642.txt", "w") as f:
    for char in metadata_df["source_name_ch1"]:
        f.write(str(char) + "\n")

In [ ]:
# Step 1: Load the annotation file
annot = pd.read_csv("unprocessed_datasets/GPL96-57554.txt", sep="\t", comment='#', low_memory=False) #ALL TEST FILES USE SAME ANNOTATION FILE

# Inspect columns
print(annot.columns.tolist())

# Step 2: Pick the relevant columns: the probe set ID (usually "ID") and the gene symbol
annot2 = annot[['ID', 'Gene Symbol']].copy()

['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']


In [48]:
#MERGE EXPRESSION MATRIX WITH PLATFORM
expr_reset = df.reset_index()
expr_reset.rename(columns={'index':'original_index'}, inplace=True)

# Ensure ID_REF is string
expr_reset['ID_REF'] = expr_reset['ID_REF'].astype(str)

# Also ensure annotation ID is string
annot2['ID'] = annot2['ID'].astype(str)

# Merge
merged = expr_reset.merge(annot2, left_on='ID_REF', right_on='ID', how='left')

print("Merged shape:", merged.shape)
print(merged[['ID_REF','ID','Gene Symbol']].head())
print(merged.head())

Merged shape: (22277, 426)
      ID_REF         ID       Gene Symbol
0  1007_s_at  1007_s_at  DDR1 /// MIR4640
1    1053_at    1053_at              RFC2
2     117_at     117_at             HSPA6
3     121_at     121_at              PAX8
4  1255_g_at  1255_g_at            GUCA1A
   original_index     ID_REF  GSM923596  GSM923597  GSM923598  GSM923599  \
0               0  1007_s_at       8.37       8.11       8.11       7.70   
1               1    1053_at       7.34       7.69       7.86       7.82   
2               2     117_at       8.25       7.10       6.93       6.60   
3               3     121_at       9.83       9.95       9.57       9.45   
4               4  1255_g_at       4.91       4.96       4.96       4.76   

   GSM923600  GSM923601  GSM923602  GSM923603  ...  GSM924010  GSM924011  \
0       7.94       8.51       8.88       8.04  ...       8.21       8.23   
1       7.62       7.90       7.18       8.00  ...       7.65       7.66   
2       7.11       7.08       6.81  

In [49]:
#REARRANGE MERGED
# List of columns to exclude (metadata)
metadata_cols = ['ID', 'Gene Symbol']

# Expression/sample columns are all other columns
expr_cols = [col for col in merged.columns if col not in metadata_cols]

# Make a new DataFrame with 'Gene Symbol' first, then expression columns
gene_expr = merged[['Gene Symbol'] + expr_cols].copy()

# Optional: check the first few rows
print(gene_expr.head())


        Gene Symbol  original_index     ID_REF  GSM923596  GSM923597  \
0  DDR1 /// MIR4640               0  1007_s_at       8.37       8.11   
1              RFC2               1    1053_at       7.34       7.69   
2             HSPA6               2     117_at       8.25       7.10   
3              PAX8               3     121_at       9.83       9.95   
4            GUCA1A               4  1255_g_at       4.91       4.96   

   GSM923598  GSM923599  GSM923600  GSM923601  GSM923602  ...  GSM924008  \
0       8.11       7.70       7.94       8.51       8.88  ...       7.87   
1       7.86       7.82       7.62       7.90       7.18  ...       7.81   
2       6.93       6.60       7.11       7.08       6.81  ...       6.94   
3       9.57       9.45       9.77       9.55      10.26  ...       9.43   
4       4.96       4.76       4.80       4.75       5.01  ...       4.83   

   GSM924009  GSM924010  GSM924011  GSM924012  GSM924013  GSM924014  \
0       8.29       8.21       8.23     

In [50]:
#CHECK REARRANGED MERGED IS OK
# Check which columns are being used as expression columns
expr_cols = [col for col in merged.columns if col.startswith('GSM')]
print("Number of expression columns:", len(expr_cols))
print("First 10 expression columns:", expr_cols[:10])
print("Last 10 expression columns:", expr_cols[-10:])

# Verify that gene_expr has 'Gene Symbol' + all expr_cols
print("Columns in gene_expr:", gene_expr.columns.tolist()[:15], "...")  # show first 15 for readability

# Expression columns (everything except the first 3 metadata columns)
expr_cols_check = gene_expr.columns[3:]

# Check number of expression columns
print("Number of expression columns:", len(expr_cols_check))

# Check first and last 5 columns
print("First 5 expression columns:", list(expr_cols_check[:5]))
print("Last 5 expression columns:", list(expr_cols_check[-5:]))

# Optional: verify no unexpected columns slipped in
unexpected_cols = [col for col in expr_cols_check if col.startswith('Gene Symbol') or col in ['original_index', 'ID_REF']]
print("Unexpected columns in expression columns:", unexpected_cols)


Number of expression columns: 422
First 10 expression columns: ['GSM923596', 'GSM923597', 'GSM923598', 'GSM923599', 'GSM923600', 'GSM923601', 'GSM923602', 'GSM923603', 'GSM923604', 'GSM923605']
Last 10 expression columns: ['GSM924008', 'GSM924009', 'GSM924010', 'GSM924011', 'GSM924012', 'GSM924013', 'GSM924014', 'GSM924015', 'GSM924016', 'GSM924017']
Columns in gene_expr: ['Gene Symbol', 'original_index', 'ID_REF', 'GSM923596', 'GSM923597', 'GSM923598', 'GSM923599', 'GSM923600', 'GSM923601', 'GSM923602', 'GSM923603', 'GSM923604', 'GSM923605', 'GSM923606', 'GSM923607'] ...
Number of expression columns: 422
First 5 expression columns: ['GSM923596', 'GSM923597', 'GSM923598', 'GSM923599', 'GSM923600']
Last 5 expression columns: ['GSM924013', 'GSM924014', 'GSM924015', 'GSM924016', 'GSM924017']
Unexpected columns in expression columns: []


In [51]:
import numpy as np
import pickle
X_test = gene_expr[expr_cols].values.astype(np.float32)  # shape: (num_genes, num_samples) - expression matrix for GeneFormer
gene_names = gene_expr['Gene Symbol'].tolist() #Y, corresponding list of gene names

np.save("X_GSE37642.npy", X_test)
'''
# save gene_names as a pickle file
with open("gene_names.pkl", "wb") as f:
    pickle.dump(gene_names, f)
'''
#print("Files saved: X.npy and gene_names.pkl")

# save X as a text file
# each row is a gene, values separated by spaces
np.savetxt("X_GSE37642.txt", X_test.astype(np.float32), fmt="%.6f")

# save gene names as a plain text file, one gene per line
with open("gene_names.txt", "w") as f:
    for gene in gene_names:
        f.write(str(gene) + "\n")

In [53]:
metadata_df["title"].unique()

array(['AML_patient_1', 'AML_patient_2', 'AML_patient_3', 'AML_patient_4',
       'AML_patient_5', 'AML_patient_6', 'AML_patient_7', 'AML_patient_8',
       'AML_patient_9', 'AML_patient_10', 'AML_patient_11',
       'AML_patient_12', 'AML_patient_13', 'AML_patient_14',
       'AML_patient_15', 'AML_patient_16', 'AML_patient_17',
       'AML_patient_18', 'AML_patient_19', 'AML_patient_20',
       'AML_patient_21', 'AML_patient_22', 'AML_patient_23',
       'AML_patient_24', 'AML_patient_25', 'AML_patient_26',
       'AML_patient_27', 'AML_patient_28', 'AML_patient_29',
       'AML_patient_30', 'AML_patient_31', 'AML_patient_32',
       'AML_patient_33', 'AML_patient_34', 'AML_patient_35',
       'AML_patient_36', 'AML_patient_37', 'AML_patient_38',
       'AML_patient_39', 'AML_patient_40', 'AML_patient_41',
       'AML_patient_42', 'AML_patient_43', 'AML_patient_44',
       'AML_patient_45', 'AML_patient_46', 'AML_patient_47',
       'AML_patient_48', 'AML_patient_49', 'AML_patient_50

This notebook prepares gene-expression data from the GSE13159 leukemia cohort for fine-tuning Geneformer to predict leukemia subtype. We now have three required inputs: an expression matrix X (genes × samples), a list of corresponding gene_names, and metadata containing each sample’s leukemia class label. The next step is to convert each sample into a Geneformer-compatible format by ranking genes by expression, mapping gene symbols to Geneformer token IDs, and assigning each sample a numeric subtype label. These tokenized samples will be packaged into a HuggingFace Dataset of {input_ids, labels} entries, which can then be used directly to fine-tune the pre-trained Geneformer model for multiclass leukemia subtype classification.

In [90]:
# Extract subtype from "leukemia class: XYZ"
subtypes = metadata_df["characteristics_ch1"].astype(str).str.split(pat=": ", n=1).str[1]

# Build mapping: subtype string -> label ID
unique_subtypes = sorted(subtypes.unique())
label_map = {label: i for i, label in enumerate(unique_subtypes)}
id_to_label = {i: label for label, i in label_map.items()} #not updated in terms of indices

labels = np.array([label_map[s] for s in subtypes], dtype=np.int64)

# categorize
# ALL 0-2 -> 0
# AML 3-8 -> 3
# CLL 9 -> 9
# CML 10 -> 10
# MDS 11 -> 11
# healthy 12 -> 12
# pro-B-ALL 13 -> 13
# T-ALL 14 -> 14
# c-ALL/Pre-B-ALL 15-16 -> 15
# mature B-ALL 17 -> 17
#labels = np.array([0,0,0,3,3,3,3,3,3,9,10,11,12,13,14,15,15,17], dtype=np.int64)
'''
merge_map = {
    0: 0, 1: 0, 2: 0,          # ALL 0-2 -> 0
    3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3,  # AML 3-8 -> 3
    9: 9,                       # CLL
    10: 10,                     # CML
    11: 11,                     # MDS
    12: 12,                     # healthy
    13: 13,                     # pro-B-ALL
    14: 14,                     # T-ALL
    15: 15, 16: 15,             # c-ALL/Pre-B-ALL
    17: 17                      # mature B-ALL
}
# Remap the labels according to merge_map
merged_labels = np.array([merge_map[l] for l in labels], dtype=np.int64)
'''
merge_map2 = {
    0: 0, 1: 0, 2: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 12: 0,       
    3: 3, 4: 3, 5: 3, 6: 3, 7: 3, 8: 3,                    
   # 12: 12,                                   
}
# categorize
# leukemia non-AML 0,1,2,9,10,11,13,14,15,16,17 = 0
# AML 3,4,5,6,7,8 = 3
# healthy 12 = 12

# Remap the labels according to merge_map
merged_labels = np.array([merge_map2[l] for l in labels], dtype=np.int64)



In [91]:
import pickle
from pathlib import Path
import numpy as np

# 1. Load the token dictionary for the 30M model
token_dict_path = Path("Geneformer/geneformer/gene_dictionaries_30m/token_dictionary_gc30m.pkl")
with open(token_dict_path, "rb") as f:
    token_dict = pickle.load(f)

# normalize keys to uppercase for robustness
token_dict = {k.upper(): v for k, v in token_dict.items()}
print("Token dict entries:", len(token_dict))

# 2. Rearrange X: samples x genes
X_samples = X.T   # was (genes, samples) -> now (samples, genes)
print("X_samples shape:", X_samples.shape)  # should be (num_samples, num_genes)

# 3. Convert one sample to a ranked token sequence
def convert_sample_to_geneformer_tokens(expr_vector, gene_names, token_dict, max_len=2048):
    # rank genes by expression (descending)
    ranked_idx = expr_vector.argsort()[::-1]

    tokens = []
    for idx in ranked_idx:
        gene = str(gene_names[idx]).upper()
        if gene in token_dict:
            tokens.append(token_dict[gene])
        if len(tokens) == max_len:
            break
    return tokens

# 4. Convert ALL samples
tokenized_samples = [
    convert_sample_to_geneformer_tokens(X_samples[i], gene_names, token_dict)
    for i in range(X_samples.shape[0])
]

print("Number of samples tokenized:", len(tokenized_samples))
print("Example tokenized sample (first 20 tokens):", tokenized_samples[0][:20])


Token dict entries: 25426
X_samples shape: (2096, 54675)
Number of samples tokenized: 2096
Example tokenized sample (first 20 tokens): []


In [92]:
# 1. token_dictionary: gene_id -> token_id
with open("Geneformer/geneformer/gene_dictionaries_30m/token_dictionary_gc30m.pkl", "rb") as f:
    id_to_token = pickle.load(f)

# 2. gene_name_id_dict: gene_symbol -> gene_id
with open("Geneformer/geneformer/gene_dictionaries_30m/gene_name_id_dict_gc30m.pkl", "rb") as f:
    name_to_id = pickle.load(f)

print("len(id_to_token):", len(id_to_token))
print("len(name_to_id):", len(name_to_id))

# normalize keys to uppercase for robustness
name_to_id_upper = {k.upper(): v for k, v in name_to_id.items()}

gene_to_token = {}
missing_in_token = 0

for name_up, gene_id in name_to_id_upper.items():
    if gene_id in id_to_token:
        gene_to_token[name_up] = id_to_token[gene_id]
    else:
        missing_in_token += 1

print("gene_to_token size:", len(gene_to_token))
print("name_ids missing from token dict:", missing_in_token)
gene_names_up = [str(g).upper() for g in gene_names]
overlap = len(set(gene_names_up) & set(gene_to_token.keys()))
print("Overlap between your genes and Geneformer vocab:", overlap)
def convert_sample_to_geneformer_tokens(expr_vector, gene_names, gene_to_token, max_len=2048):
    # rank genes by expression (descending)
    ranked_idx = expr_vector.argsort()[::-1]
    tokens = []
    for idx in ranked_idx:
        gene = str(gene_names[idx]).upper()
        if gene in gene_to_token:
            tokens.append(gene_to_token[gene])
        if len(tokens) == max_len:
            break
    return tokens
X_samples = X.T  # samples x genes

tokenized_samples = [
    convert_sample_to_geneformer_tokens(X_samples[i], gene_names, gene_to_token)
    for i in range(X_samples.shape[0])
]

import numpy as np
lengths = np.array([len(seq) for seq in tokenized_samples])
print("Min length:", lengths.min())
print("Max length:", lengths.max())
print("Number of empty sequences:", np.sum(lengths == 0))



len(id_to_token): 25426
len(name_to_id): 40248
gene_to_token size: 21245
name_ids missing from token dict: 19003
Overlap between your genes and Geneformer vocab: 16047
Min length: 2048
Max length: 2048
Number of empty sequences: 0


In [94]:
import numpy as np

PAD_ID = 0
MAX_LEN = 128

input_ids = []
attention_masks = []

for seq in tokenized_samples:
    seq = seq[:MAX_LEN]
    pad_len = MAX_LEN - len(seq)
    padded = seq + [PAD_ID] * pad_len
    mask = [1] * len(seq) + [0] * pad_len

    input_ids.append(padded)
    attention_masks.append(mask)

input_ids = np.array(input_ids, dtype=np.int64)
attention_masks = np.array(attention_masks, dtype=np.int64)

print("input_ids shape:", input_ids.shape)
print("attention_masks shape:", attention_masks.shape)
print("labels shape:", labels.shape)
print("merged labels shape:", merged_labels.shape)


input_ids shape: (2096, 128)
attention_masks shape: (2096, 128)
labels shape: (2096,)
merged labels shape: (2096,)


In [95]:
from transformers import BertForSequenceClassification
import numpy as np

model_name = "ctheodoris/Geneformer"

model = BertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(np.unique(labels)),  # your 18 leukemia classes
)

print("Loaded model:", model_name)
print("Pad token id:", model.config.pad_token_id)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ctheodoris/Geneformer and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded model: ctheodoris/Geneformer
Pad token id: 0


In [96]:
from pathlib import Path
import pickle

dict_base = Path("Geneformer/geneformer/gene_dictionaries_30m")

with open(dict_base / "token_dictionary_gc30m.pkl", "rb") as f:
    token_dict = pickle.load(f)

token_dict = {k.upper(): v for k, v in token_dict.items()}


In [97]:
from datasets import Dataset

ds = Dataset.from_dict({
    "input_ids": input_ids,
    "attention_mask": attention_masks,
    "labels": merged_labels, #was labels
})

ds_split = ds.train_test_split(test_size=0.2, seed=42)
train_ds = ds_split["train"]
eval_ds  = ds_split["test"]

small_train_ds = train_ds #.select(range(min(64, len(train_ds))))
small_eval_ds  = eval_ds #.select(range(min(32, len(eval_ds))))

print("Small train size:", len(small_train_ds))
print("Small eval size:", len(small_eval_ds))

import transformers
from transformers import Trainer

TA = transformers.TrainingArguments

training_args = TA(
    output_dir="geneformer_leukemia_progress",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    logging_steps=5,
)


Small train size: 1676
Small eval size: 420


In [ ]:
for label, idx in label_map.items():
    print(f"{label}: {idx}")

# categorize
# ALL 0-2 -> 0
# AML 3-8 -> 3
# CLL 9 -> 9
# CML 10 -> 10
# MDS 11 -> 11
# healthy 12 -> 12
# pro-B-ALL 13 -> 13
# T-ALL 14 -> 14
# c-ALL/Pre-B-ALL 15-16 -> 15
# mature B-ALL 17 -> 17


ALL with hyperdiploid karyotype: 0
ALL with t(12;21): 1
ALL with t(1;19): 2
AML complex aberrant karyotype: 3
AML with inv(16)/t(16;16): 4
AML with normal karyotype + other abnormalities: 5
AML with t(11q23)/MLL: 6
AML with t(15;17): 7
AML with t(8;21): 8
CLL: 9
CML: 10
MDS: 11
Non-leukemia and healthy bone marrow: 12
Pro-B-ALL with t(11q23)/MLL: 13
T-ALL: 14
c-ALL/Pre-B-ALL with t(9;22): 15
c-ALL/Pre-B-ALL without t(9;22): 16
mature B-ALL with t(8;14): 17


In [99]:
import torch

def data_collator(features):
    input_ids = torch.tensor([f["input_ids"] for f in features], dtype=torch.long)
    attention_mask = torch.tensor([f["attention_mask"] for f in features], dtype=torch.long)
    labels_batch = torch.tensor([f["labels"] for f in features], dtype=torch.long)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels_batch,
    }
import transformers
from transformers import Trainer

TA = transformers.TrainingArguments
'''
training_args = TA(
    output_dir="geneformer_leukemia_progress",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_steps=10,
)
'''

training_args = TA(
    output_dir="geneformer_leukemia_progress",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=10,

    #evaluation_strategy="steps",
    #eval_steps=50,
    #save_strategy="no",
)

trainer = Trainer(
    model=model,  # BertForSequenceClassification from "ctheodoris/Geneformer"
    args=training_args,
    train_dataset=small_train_ds,
    eval_dataset=small_eval_ds,
    data_collator=data_collator,
)

trainer.train()


/Users/charlottecheung/miniforge3/envs/geneformer_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,1.165000
20,0.631900
30,0.638600
40,0.611100
50,0.391200
60,0.471000
70,0.563300
80,0.396400
90,0.265000
100,0.304100


TrainOutput(global_step=210, training_loss=0.42610773586091544, metrics={'train_runtime': 179.34, 'train_samples_per_second': 9.345, 'train_steps_per_second': 1.171, 'total_flos': 371058800560128.0, 'train_loss': 0.42610773586091544, 'epoch': 1.0})

In [100]:
import numpy as np
from sklearn.metrics import classification_report

preds = trainer.predict(small_eval_ds)
logits = preds.predictions
y_pred = np.argmax(logits, axis=1)
y_true = preds.label_ids

# only the labels that actually appear in this eval split
labels_present = np.unique(y_true)
target_names = [id_to_label[i] for i in labels_present]

print("Labels present in eval:", labels_present)
print("Target names used:", target_names)

print(classification_report(
    y_true,
    y_pred,
    labels=labels_present,
    target_names=target_names,
))
import collections
print("True counts:", collections.Counter(y_true))
print("Pred counts:", collections.Counter(y_pred))



/Users/charlottecheung/miniforge3/envs/geneformer_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Labels present in eval: [0 3]
Target names used: ['ALL with hyperdiploid karyotype', 'AML complex aberrant karyotype']
                                 precision    recall  f1-score   support

ALL with hyperdiploid karyotype       0.94      0.94      0.94       320
 AML complex aberrant karyotype       0.82      0.82      0.82       100

                       accuracy                           0.91       420
                      macro avg       0.88      0.88      0.88       420
                   weighted avg       0.91      0.91      0.91       420

True counts: Counter({np.int64(0): 320, np.int64(3): 100})
Pred counts: Counter({np.int64(0): 320, np.int64(3): 100})
